In [2]:
!pip install pandas dash
!pip install jupyter_dash

     |████████████████████████████████| 7.3 MB 5.2 MB/s 
     |████████████████████████████████| 25.3 MB 37.6 MB/s 
     |████████████████████████████████| 357 kB 51.4 MB/s 
  Created wheel for dash-core-components: filename=dash_core_components-2.0.0-py3-none-any.whl size=3821 sha256=c627d9fc705b64b2db92495ff89ef8ee25b4d5e11d915837310672d1d51ad66b
  Stored in directory: /root/.cache/pip/wheels/00/f9/c7/1a6437d794ed753ea9bc9079e761d4fc803a1f1f5d3697b9ec
  Created wheel for dash-html-components: filename=dash_html_components-2.0.0-py3-none-any.whl size=4089 sha256=493e3d4f35762ca48e012524e670c036d36e10fbe44cc695db5c7342230ab20b
  Stored in directory: /root/.cache/pip/wheels/ec/6b/81/05aceabd8b27f724e2c96784016287cc1bfbc349ebfda451de
  Created wheel for dash-table: filename=dash_table-5.0.0-py3-none-any.whl size=3911 sha256=bf32e41cf9b0c2d74144e6af3524f49ddf4a67c394bbcd99bfc64ed1ca5e88f5
  Stored in directory: /root/.cache/pip/wheels/85/5d/4e/7c276b57992951dbe770bf5caad6448d0539c510663ae

In [3]:
!wget "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_dash.csv"

--2021-11-24 20:03:46--  https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_dash.csv
Resolving cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)... 169.63.118.104
Connecting to cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)|169.63.118.104|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3033 (3.0K) [text/csv]
Saving to: ‘spacex_launch_dash.csv’

spacex_launch_dash. 100%[===================>]   2.96K  --.-KB/s    in 0s      

2021-11-24 20:03:47 (364 MB/s) - ‘spacex_launch_dash.csv’ saved [3033/3033]



In [73]:
# Import required libraries
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
import dash
#import dash_html_components as html
from dash import html
#import dash_core_components as dcc
from dash import dcc
from dash.dependencies import Input, Output, State
from jupyter_dash import JupyterDash
#import plotly.graph_objects as go
#import plotly.express as px
from dash import no_update

# Read the airline data into pandas dataframe
spacex_df = pd.read_csv("spacex_launch_dash.csv")
max_payload = spacex_df['Payload Mass (kg)'].max()
min_payload = spacex_df['Payload Mass (kg)'].min()

# Create a dash application
#app = dash.Dash(__name__)
app = JupyterDash(__name__)
JupyterDash.infer_jupyter_proxy_config()

# Create an app layout
app.layout = html.Div(children=[html.H1('SpaceX Launch Records Dashboard',
                                        style={'textAlign': 'center', 'color': '#503D36',
                                               'font-size': 40}),
                                # TASK 1: Add a dropdown list to enable Launch Site selection
                                # The default select value is for ALL sites
                                html.Div([
                                 dcc.Dropdown(id='site-dropdown',
                                              options=[
                                                              {'label':'All Sites','value':'ALL'},
                                                              {'label':'CCAFS LC-40','value':'CCAFS LC-40'},
                                                              {'label':'VAFB SLC-4E','value':'VAFB SLC-4E'}, 
                                                              {'label':'KSC LC-39A','value':'KSC LC-39A'}, 
                                                              {'label':'CCAFS SLC-40','value':'CCAFS SLC-40'}, 
                                                     ],
                                                     value ="ALL",
                                                     placeholder="Select a Launch site here",
                                                     searchable = True,
                                                     style={'width':'80%', 'padding':'3px', 'font-size': '20px', 'text-align-last' : 'center'}
                                              )
                                ]),
                              #  html.Br(),

                                # TASK 2: Add a pie chart to show the total successful launches count for all sites
                                # If a specific launch site was selected, show the Success vs. Failed counts for the site
                                html.Div(dcc.Graph(id='success-pie-chart')),
                              #  html.Br(),

                                html.P("Payload range (Kg):"),
                                # TASK 3: Add a slider to select payload range
                                #dcc.RangeSlider(id='payload-slider',...)
                           #     html.Br(),
                                html.Div(
                                dcc.RangeSlider(id='payload-slider',
                                              min=0, max=10000, step=1000,
                                              marks={0: '0',
                                              100: '100'},
                                               value=[min_payload, max_payload],
                                                tooltip={"placement": "bottom", "always_visible": True})
                                ),
                               # html.Br(),
                                # TASK 4: Add a scatter chart to show the correlation between payload and launch success
                                html.Div(dcc.Graph(id='success-payload-scatter-chart')),
                                ])

# TASK 2:
# Add a callback function for `site-dropdown` as input, `success-pie-chart` as output
# Function decorator to specify function input and output
@app.callback(Output(component_id='success-pie-chart', component_property='figure'),
              Input(component_id='site-dropdown', component_property='value'))
def get_pie_chart(entered_site):
    filtered_df = spacex_df
    if entered_site == 'ALL':
        fig = px.pie(filtered_df, values='class', 
        names='Launch Site', 
        title='Total Success Launches By Site')
        return fig
    else:
        # return the outcomes piechart for a selected site
        filtered_df=spacex_df[spacex_df['Launch Site']==entered_site]
        fig = px.pie(filtered_df, #values='class', 
        names='class', 
        title='Total Success Launches for Site '+ entered_site)
        return fig



# TASK 4:
# Add a callback function for `site-dropdown` and `payload-slider` as inputs, `success-payload-scatter-chart` as output
@app.callback(Output(component_id='success-payload-scatter-chart', component_property='figure'),
              [Input(component_id='site-dropdown', component_property='value'), 
               Input(component_id="payload-slider", component_property="value")])
def get_scatter_chart(entered_site,entered_payload):
    filtered_df = spacex_df
    if entered_site == 'ALL':
        fig = px.scatter(filtered_df, x='Payload Mass (kg)', 
        y='class', 
        color="Booster Version Category",
        title='Correlation between Payload and Success for all Sites')
        return fig
    else:
        # return the outcomes piechart for a selected site
        filtered_df=spacex_df[spacex_df['Launch Site']==entered_site ]
      #  filtered_df=filtered_df[filtered_df['Payload Mass (kg)']>=entered_payload[0] ]
      #  filtered_df=filtered_df[filtered_df['Payload Mass (kg)']<=entered_payload[1] ]

        fig = px.scatter(filtered_df, x='Payload Mass (kg)', 
        y='class', 
        color="Booster Version Category",
        title='Correlation between Payload and Success for Site '+entered_site)
        return fig


# Run the app
if __name__ == '__main__':
    app.run_server()

Dash app running on:


<IPython.core.display.Javascript object>

In [27]:
min_payload,max_payload

(0.0, 9600.0)

In [50]:
spacex_df.head(5)

,Unnamed: 0,Flight Number,Launch Site,Mission Outcome,class,Payload Mass (kg),Booster Version,Booster Version Category
0,0,1,CCAFS LC-40,Success,0,0.0,F9 v1.0 B0003,v1.0
1,1,2,CCAFS LC-40,Success,0,0.0,F9 v1.0 B0004,v1.0
2,2,3,CCAFS LC-40,Success,0,525.0,F9 v1.0 B0005,v1.0
3,3,4,CCAFS LC-40,Success,0,500.0,F9 v1.0 B0006,v1.0
4,4,5,CCAFS LC-40,Success,0,677.0,F9 v1.0 B0007,v1.0


In [68]:
spacex_df[spacex_df['Payload Mass (kg)']>=30 ].head()

,Unnamed: 0,Flight Number,Launch Site,Mission Outcome,class,Payload Mass (kg),Booster Version,Booster Version Category
2,2,3,CCAFS LC-40,Success,0,525.0,F9 v1.0 B0005,v1.0
3,3,4,CCAFS LC-40,Success,0,500.0,F9 v1.0 B0006,v1.0
4,4,5,CCAFS LC-40,Success,0,677.0,F9 v1.0 B0007,v1.0
5,5,7,CCAFS LC-40,Success,0,3170.0,F9 v1.1,v1.1
6,6,8,CCAFS LC-40,Success,0,3325.0,F9 v1.1,v1.1


In [55]:
spacex_df[spacex_df.class==0].head()

SyntaxError: ignored

In [49]:
filtered_df=spacex_df[spacex_df['Payload Mass (kg)'] >0 and spacex_df['Payload Mass (kg)']<5000 ]
filtered_df.head()

ValueError: ignored

In [24]:
px.pie(filtered_df, #values='class', 
        names='class', 
        title='Total Success Launches for Site ')